### Whisp a geojson

Setup
- Use a [virtual environment](https://docs.python.org/3/tutorial/venv.html) to avoid altering your python environment 

Usage:
- Use this notebook with smaller datasets (e.g., up to 10,000 features). 
- For larger datasets consider the 'whisp_geojson_to_drive.ipynb' notebook, which is more suited to heavy processing
- Please report issues with this notebook [here](https://github.com/forestdatapartnership/whisp/issues)

In [ ]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine. 
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate() # Authenticate may open a browser window
    ee.Initialize()

# NB if not working add your cloud project: ee.Initialize(project="your_gee_cloud_project_name")

In [ ]:
# Install openforis-whisp (uncomment line if not already installed)
# !pip install --pre openforis-whisp

# NB for editable mode install via your terminal with: pip install -e .[dev]

In [ ]:
import openforis_whisp as whisp

Get a geojson

In [ ]:
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")

Prepare inputs

In [ ]:
# Choose if want to include additional custom layers
USE_CUSTOM_BANDS = False # set to True if want to add extra ee data to whisp

In [ ]:
# =============================================================================
# CUSTOM BANDS SETUP (OPTIONAL) - runs only if USE_CUSTOM_BANDS = True above
# =============================================================================
if USE_CUSTOM_BANDS:

    # Step 1: Define custom Earth Engine images (binary values 0 or 1)
    custom_images = {
        'example_treecover': ee.Image(1),  # ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select("treecover2000").gt(10).selfMask()
        'nXX_example_commodity': ee.Image.random(seed=1).gte(.5).reproject(crs='EPSG:4326', scale=10) # ee.ImageCollection("projects/forestdatapartnership/assets/cocoa/model_2025a").filter(ee.Filter.date('2020-01-01', '2021-01-01')).mosaic().gt(.8).selfMask()
        # add more images as needed (prefix 'nXX_' = iso2 code for national dataset)
    }

    # Step 2: Define metadata for each custom band (keys must match above)
    # Themes: 'treecover', 'commodities', 'disturbance_before', 'disturbance_after'
    # Timber themes: 'primary', 'naturally_reg_2020', 'planted_plantation_2020', etc.
    custom_bands_info = {
        'example_treecover': {
            'ISO2_code': "",          # Country code (empty = all countries)
            'theme': 'treecover',     # Risk theme
            'theme_timber': "",       # Timber theme (if applicable)
            'use_for_risk': 1,        # Include in risk calculations (1=yes, 0=no)
            'use_for_risk_timber': 0  # Include in timber risk (1=yes, 0=no)
        },
        'nXX_example_commodity': {
            'ISO2_code': "XX", 
            'theme': 'commodities', 
            'theme_timber': "",
            'use_for_risk': 1, 
            'use_for_risk_timber': 0
        }
        # add more band metadata as needed
    }

    # Step 3: Combine custom bands and extract names
    custom_ee_image = whisp.combine_custom_bands(custom_images, custom_bands_info)

    custom_bands = list(custom_bands_info.keys())


In [ ]:
# Choose additional national datasets to include (currently three countries: 'co', 'ci', 'br').
base_iso2_codes = ['co', 'ci', 'br']

# automatically add any custom ISO2 codes from custom_bands_info if USE_CUSTOM_BANDS is True
iso2_codes_list = base_iso2_codes.copy()
if USE_CUSTOM_BANDS:
    iso2_codes_list += [code.lower() for code in {v.get('ISO2_code') for v in custom_bands_info.values()} if code and code.lower() not in iso2_codes_list]

In [ ]:
# Create final Whisp image
whisp_image = whisp.combine_datasets(national_codes=iso2_codes_list)
standard_bands = len(whisp_image.bandNames().getInfo())

if USE_CUSTOM_BANDS and 'custom_ee_image' in locals():
    whisp_image = whisp_image.addBands(custom_ee_image)
    print(f"Final image has {standard_bands + len(custom_bands)} bands ({standard_bands} + {len(custom_bands)} custom)")
else:
    print(f"Final image has {standard_bands} bands")

Run Whisp 

In [ ]:
df_stats = whisp.whisp_formatted_stats_geojson_to_df(
    input_geojson_filepath=GEOJSON_EXAMPLE_FILEPATH,
    # external_id_column="user_id",  # optional - specify which input column/property to map to the external ID.
    national_codes=iso2_codes_list,  # optional - By default national datasets are not included unless specified here.
    # unit_type='percent',  # optional - to change unit type. Default is 'ha'.
    whisp_image=whisp_image,  # optional - defaults to standard whisp image if not provided
    custom_bands=custom_bands if USE_CUSTOM_BANDS else None  # include custom bands in formatted output
)

Display results

In [ ]:
df_stats

In [ ]:
# Define the output folder (if running in Sepal change path to preferred folder) 
# e.g. out_directory = Path.home() / 'module_results/whisp/'
out_directory = Path.home() / 'downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table.csv'

# Save the CSV file
df_stats.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table saved to: {csv_output_file}")

Calculate risk category

In [ ]:
# adds risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(
    df=df_stats,
    national_codes=iso2_codes_list,
    custom_bands_info=custom_bands_info if USE_CUSTOM_BANDS else None  # Add: missing custom bands
)

Display table with risk columns

In [ ]:
df_w_risk

Export table to CSV

In [ ]:
# Define the output folder 
# e.g. in running in Sepal this might be: Path.home() / 'module_results/whisp/'
out_directory = Path.home() / 'downloads'

# Define the output file path for CSV
csv_output_file = out_directory / 'whisp_output_table_w_risk.csv'

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
geojson_output_file = out_directory / 'whisp_output_geo_w_risk.geojson'

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.